## Install Cobrapy Package

In [0]:
!pip install cobra

# Simulating Deletions

In [0]:
import pandas
from time import time

import cobra.test
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

cobra_model = cobra.test.create_test_model("textbook")
biomass_rxn = cobra_model.reactions.get_by_id('Biomass_Ecoli_core')
biomass_rxn.id = 'BIOMASS_Ecoli_core_w_GAM' 

ecoli_model = cobra.test.create_test_model("ecoli")

## Knocking out single genes and reactions

A commonly asked question when analyzing metabolic models is what will happen if a certain reaction was not allowed to have any flux at all. This can tested using cobrapy by

In [0]:
solution_1 = cobra_model.optimize()
print('complete model: ', solution_1)
with cobra_model:
    cobra_model.reactions.PFK.knock_out()
    solution_2 = cobra_model.optimize()
    print('pfk knocked out: ', solution_2)

In [0]:
cobra_model.reactions.get_by_id('PFK')

In [0]:
import os
from google.colab import files

directory = 'results_deletions'
if not os.path.exists(directory):
  os.mkdir(directory)

solution_1.fluxes.to_json('results_deletions/flux_del_1.json')
files.download('results_deletions/flux_del_1.json')

solution_2.fluxes.to_json('results_deletions/flux_del_2.json')
files.download('results_deletions/flux_del_2.json')


Load flux solution in [Escher Maps](https://escher.github.io) using the *Core metabolism* map.

For evaluating genetic manipulation strategies, it is more interesting to examine what happens if given genes are knocked. A knockout might have no effect at all in case of redundancy, or affect several reactions if the gene is participating in more than one reaction.

In [0]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.genes.b1723.knock_out()
    print('pfkA knocked out: ', cobra_model.optimize())
    cobra_model.genes.b3916.knock_out()
    print('pfkB knocked out: ', cobra_model.optimize())

## Single Deletions

Perform all single gene deletions on a model

In [0]:
deletion_results = single_gene_deletion(cobra_model)

These can also be done for only a subset of genes

In [0]:
single_gene_deletion(cobra_model, cobra_model.genes[:20])

This can also be done for reactions

In [0]:
single_reaction_deletion(cobra_model, cobra_model.reactions[:20])

## Double Deletions

Double deletions run in a similar way. Passing in `return_frame=True` will cause them to format the results as a `pandas.DataFrame`.

In [0]:
double_gene_deletion(cobra_model, cobra_model.genes[-5:], return_frame=True).round(4)

By default, the double deletion function will automatically use multiprocessing, splitting the task over up to 4 cores if they are available. The number of cores can be manually specified as well. Setting use of a single core will disable use of the multiprocessing library, which often aids debugging.

Double deletions can also be run for reactions.

In [0]:
double_reaction_deletion(cobra_model, cobra_model.reactions[2:7], return_frame=True).round(4)